# ADVANCED PANDAS

In [91]:
import numpy as np
import pandas as pd

In [3]:
# Background and Motivation


# Frequently, a column in a table may contain repeated instances of a smaller set of distinct values.
    # We have already seen functions like unique and value_counts, 
    # which enable us to extract the distinct values from an array and compute their frequencies, respectively:
    
import numpy as np; import pandas as pd


values = pd.Series(['apple', 'orange', 'apple','apple'] * 2)

values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [4]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [5]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [7]:
# Many data systems (for data warehousing, statistical computing, or other uses) have developed specialized
    # approaches for representing data with repeated values for more efficient storage and computation. 
    # In data warehousing, a best practice is to use socalled dimension tables containing the distinct values 
    # and storing the primary observations as integer keys referencing the dimension table:

values = pd.Series([0, 1, 0, 0] * 2)

dim = pd.Series(['apple', 'orange'])

In [8]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [9]:
dim

0     apple
1    orange
dtype: object

In [11]:
# We can use the take method to restore the original Series of strings:

dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

In [ ]:
# This representation as integers is called the categorical or dictionary-encoded representation.
    # The array of distinct values can be called the categories, dictionary, or levels of the data. 
    # The integer values that reference the categories are called the category codes or simply codes.

# The categorical representation can yield significant performance improvements when you are doing analytics.
    # You can also perform transformations on the categories while leaving the codes unmodified.
    # Some example transformations that can be made at relatively low cost are:

#    • Renaming categories

#    • Appending a new category without changing the order or position of the existing categories

In [12]:
# Categorical Type in pandas

# pandas has a special Categorical type for holding data that uses the integer-based 
    # categorical representation or encoding. 

# Let’s consider the example Series from before:


fruits = ['apple', 'orange', 'apple', 'apple'] * 2

N = len(fruits)

df = pd.DataFrame({'fruit': fruits, 
                   'basket_id': np.arange(N), 
                   'count': np.random.randint(3, 15, size=N), 
                   'weight': np.random.uniform(0, 4, size=N)}, 
                  columns=['basket_id', 'fruit', 'count', 'weight'])


df

,basket_id,fruit,count,weight
0,0,apple,4,2.188239
1,1,orange,6,3.023703
2,2,apple,5,0.357328
3,3,apple,8,2.489471
4,4,apple,3,3.331382
5,5,orange,9,3.562300
6,6,apple,12,2.087699
7,7,apple,11,0.365287


In [14]:
df.to_excel('fruitbasket.xlsx')

In [16]:
# Above, df['fruit'] is an array of Python string objects. We can convert it to categorical by calling:

fruit_cat = df['fruit'].astype('category')

fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [17]:
# The values for fruit_cat are not a NumPy array, but an instance of pandas.Categorical:

c = fruit_cat.values
type(c)

pandas.core.arrays.categorical.Categorical

In [18]:
# The Categorical object has categories and codes attributes:

c.categories

Index(['apple', 'orange'], dtype='object')

In [19]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [20]:
# You can convert a DataFrame column to categorical by assigning the converted result:

df['fruit'] = df['fruit'].astype('category')

df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [21]:
# You can also create pandas.Categorical directly from other types of Python sequences:


my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])

my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [22]:
# If you have obtained categorical encoded data from another source, 
    # you can use the alternative from_codes constructor:

categories = ['foo', 'bar', 'baz']

codes = [0, 1, 2, 0, 0, 1]

my_cats_2 = pd.Categorical.from_codes(codes, categories)

my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [23]:
# Unless explicitly specified, categorical conversions assume no specific ordering of the categories.
    # So the categories array may be in a different order depending on the ordering of the input data.
    # When using from_codes or any of the other constructors, 
    # you can indicate that the categories have a meaningful ordering:

        
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)

ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [24]:
# The above output [foo < bar < baz] indicates that 'foo' precedes 'bar' in the ordering,and so on.
    # An unordered categorical instance can be made ordered with as_ordered:


my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [ ]:
# As a last note, categorical data need not be strings, even though I have only showed string examples.
# A categorical array can consist of any immutable value types.

In [25]:
# Computations with Categoricals

# Using Categorical in pandas compared with the non-encoded version (like an array of strings)
    # generally behaves the same way.
    # Some parts of pandas, like the groupby function, perform better when working with categoricals.
    # There are also some functions that can utilize the ordered flag.
    
#Let’s consider some random numeric data, and use the pandas.qcut binning function.
    # This return pandas.Categorical:
    
np.random.seed(12345)

draws = np.random.randn(1000)

draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [26]:
draws

array([-2.04707659e-01,  4.78943338e-01, -5.19438715e-01, -5.55730304e-01,
        1.96578057e+00,  1.39340583e+00,  9.29078767e-02,  2.81746153e-01,
        7.69022568e-01,  1.24643474e+00,  1.00718936e+00, -1.29622111e+00,
        2.74991633e-01,  2.28912879e-01,  1.35291684e+00,  8.86429341e-01,
       -2.00163731e+00, -3.71842537e-01,  1.66902531e+00, -4.38569736e-01,
       -5.39741446e-01,  4.76985010e-01,  3.24894392e+00, -1.02122752e+00,
       -5.77087303e-01,  1.24121276e-01,  3.02613562e-01,  5.23772068e-01,
        9.40277775e-04,  1.34380979e+00, -7.13543985e-01, -8.31153539e-01,
       -2.37023165e+00, -1.86076079e+00, -8.60757398e-01,  5.60145293e-01,
       -1.26593449e+00,  1.19827125e-01, -1.06351245e+00,  3.32882716e-01,
       -2.35941881e+00, -1.99542955e-01, -1.54199553e+00, -9.70735912e-01,
       -1.30703025e+00,  2.86349747e-01,  3.77984111e-01, -7.53886535e-01,
        3.31285650e-01,  1.34974221e+00,  6.98766888e-02,  2.46674110e-01,
       -1.18616011e-02,  

In [27]:
# Let’s compute a quartile binning of this data and extract some statistics:

bins = pd.qcut(draws, 4)

bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [29]:
# While useful, the exact sample quartiles may be less useful for producing a report than quartile names.
    # We can achieve this with the labels argument to qcut:

    
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [30]:
bins.codes

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3, 3, 0, 2, 2, 3, 3, 0, 1, 3, 1, 1, 2,
       3, 0, 1, 2, 2, 2, 2, 3, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 1, 0, 0,
       0, 2, 2, 0, 2, 3, 2, 2, 1, 3, 3, 0, 0, 2, 3, 1, 3, 2, 2, 3, 3, 0,
       1, 0, 1, 0, 0, 3, 3, 3, 3, 1, 1, 0, 0, 2, 2, 0, 3, 2, 3, 3, 0, 3,
       1, 3, 2, 3, 1, 3, 2, 3, 2, 0, 2, 2, 0, 1, 1, 0, 1, 1, 3, 3, 1, 3,
       1, 2, 3, 0, 0, 1, 2, 1, 3, 0, 0, 0, 2, 3, 1, 1, 2, 1, 1, 1, 0, 0,
       3, 3, 2, 2, 3, 1, 1, 2, 0, 3, 2, 1, 2, 1, 1, 1, 3, 3, 0, 3, 1, 0,
       2, 0, 1, 3, 3, 1, 1, 1, 0, 3, 0, 3, 1, 3, 3, 3, 3, 1, 3, 0, 0, 1,
       0, 0, 2, 1, 1, 1, 0, 3, 3, 3, 0, 1, 1, 1, 0, 0, 3, 2, 3, 3, 0, 0,
       3, 2, 3, 1, 2, 1, 0, 0, 0, 1, 0, 1, 3, 2, 1, 3, 1, 3, 0, 2, 3, 1,
       1, 2, 0, 0, 2, 3, 3, 2, 1, 2, 0, 0, 0, 3, 1, 0, 0, 2, 2, 2, 0, 1,
       3, 1, 3, 0, 2, 1, 3, 1, 3, 0, 1, 0, 2, 0, 1, 1, 0, 2, 0, 3, 1, 3,
       0, 0, 1, 2, 1, 2, 2, 1, 0, 2, 2, 1, 0, 0, 1, 1, 1, 1, 2, 0, 3, 0,
       0, 3, 0, 2, 1, 2, 0, 2, 2, 1, 3, 3, 1, 3, 3,

In [31]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [35]:
# The labeled bins categorical does not contain information about the bin edges in the data, 
    # so we can use groupby to extract some summary statistics:

    
bins = pd.Series(bins, name='quartile')

results = (pd.Series(draws).groupby(bins).agg(['count','min', 'max']).reset_index())

results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [38]:
results2 = (pd.Series(draws).groupby(bins).agg(['count', 'std', 'var','min', 'max', 'sum']).reset_index())

results2

,quartile,count,std,var,min,max,sum
0,Q1,250,0.450448,0.202903,-2.949343,-0.685484,-303.995226
1,Q2,250,0.191283,0.036589,-0.683066,-0.010115,-90.605646
2,Q3,250,0.188923,0.035692,-0.010032,0.628894,76.960037
3,Q4,250,0.518827,0.269181,0.634238,3.927528,315.290125


In [39]:
# The 'quartile' column in the result retains the original categorical information, 
    # including ordering, from bins:


results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [42]:
# Better performance with categoricals

# If you do a lot of analytics on a particular dataset, converting to categorical can yield
    # substantial overall performance gains. 
    # A categorical version of a DataFrame column will often use significantly less memory, too. 

# Let’s consider some Series with 10 million elements and a small number of distinct categories:


N = 10000000

draws = pd.Series(np.random.randn(N))

labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

In [43]:
draws

0         -0.991821
1          0.840305
2          1.242114
3          0.217027
4         -0.427622
             ...   
9999995    0.606146
9999996   -1.507969
9999997   -0.319387
9999998    1.338074
9999999    0.739566
Length: 10000000, dtype: float64

In [44]:
labels

0          foo
1          bar
2          baz
3          qux
4          foo
          ... 
9999995    qux
9999996    foo
9999997    bar
9999998    baz
9999999    qux
Length: 10000000, dtype: object

In [45]:
# Now we convert labels to categorical:


categories = labels.astype('category')

In [57]:
# Now we note that labels uses significantly more memory than categories:

    
labels.memory_usage()

80000128

In [58]:
categories.memory_usage()

10000320

In [59]:
# The conversion to category is not free, of course, but it is a one-time cost:


%time _ = labels.astype('category')

Wall time: 1.19 s


In [ ]:
# GroupBy operations can be significantly faster with categoricals because the underlying
    # algorithms use the integer-based codes array instead of an array of strings.

In [60]:
# Categorical Methods

# Series containing categorical data have several special methods similar to the Series.str 
    # specialized string methods.  This also provides convenient access to the categoriesand codes. 

# Consider the Series:

    
s = pd.Series(['a', 'b', 'c', 'd'] * 2)

cat_s = s.astype('category')

cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [61]:
# The special attribute cat provides access to categorical methods:

cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [62]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [63]:
# Suppose that we know the actual set of categories for this data extends beyond the
    # four values observed in the data. We can use the set_categories method to change them:
    
actual_categories = ['a', 'b', 'c', 'd', 'e']

cat_s2 = cat_s.cat.set_categories(actual_categories)

cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [64]:
# While it appears that the data is unchanged, 
    # the new categories will be reflected in operations that use them.
    # For example, value_counts respects the categories, if present:

cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [67]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [68]:
# In large datasets, categoricals are often used as a convenient tool for memory savings
    # and better performance. 

# After you filter a large DataFrame or Series, many of the categories may not appear in the data.
    # To help with this, we can use the remove_unused_categories method to trim unobserved categories:

    
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]

cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [69]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

In [ ]:
# Categorical methods for Series in pandas


  #              Method                        Description
    
    
  #              add_categories                Append new (unused) categories at end of existing categories

  #              as_ordered                    Make categories ordered
    
  #              as_unordered                  Make categories unordered

  #              remove_categories             Remove categories, setting any removed values to null
    
  #              remove_unused_categories      Remove any category values which do not appear in the data

  #              rename_categories             Replace categories with indicated set of new category names;
  #                                              cannot change the number of categories

  #              reorder_categories            Behaves like rename_categories, but can also change 
  #                                              the result to have ordered categories

  #              set_categories                Replace the categories with the indicated set of new categories;
  #                                              can add or remove categories

In [70]:
# When you’re using statistics or machine learning tools, 
    # you’ll often transform categorical data into dummy variables, also known as one-hot encoding.
    # This involves creating a DataFrame with a column for each distinct category;
    # these columns contain 1s for occurrences of a given category and 0 otherwise.
    
#Consider the previous example:


cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

In [72]:
#The pandas.get_dummies function converts this one-dimensional categorical data into a DataFrame 
    # containing the dummy variable:
    
    
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


In [73]:
# Advanced GroupBy Use

# Group Transforms and “Unwrapped” GroupBys

# Apart from the apply method in grouped operations for performing transformations,
    # there is another built-in method called transform, which is similar to apply but imposes more constraints
    # on the kind of function you can use:

#        • It can produce a scalar value to be broadcast to the shape of the group
#        • It can produce an object of the same shape as the input group
#        • It must not mutate its input


# Let’s consider a simple example for illustration:
    
    
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4, 'value': np.arange(12.)})

df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [75]:
# Here are the group means by key:

    
g = df.groupby('key').value

g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [76]:
# Suppose instead we wanted to produce a Series of the same shape as df['value'] 
    # but with values replaced by the average grouped by 'key'.
    # We can pass the function lambda x: x.mean() to transform:

g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [77]:
# For built-in aggregation functions, we can pass a string alias as with the GroupBy agg method:

g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [78]:
# Like apply, transform works with functions that return Series, 
    # but the result must be the same size as the input.
    # For example, we can multiply each group by 2 using a lambda function:

    
g.transform(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [79]:
# As a more complicated example, we can compute the ranks in descending order for each group:

g.transform(lambda x: x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [80]:
# Consider a group transformation function composed from simple aggregations:

    
def normalize(x):
    return (x - x.mean()) / x.std()

In [81]:
# We can obtain equivalent results in this case either using transform or apply:

g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [82]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [83]:
# Built-in aggregate functions like 'mean' or 'sum' are often much faster than a general apply function.
    # These also have a “fast past” when used with transform.
    # This allows us to perform a so-called unwrapped group operation:
    
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [84]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')

normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [ ]:
# While an unwrapped group operation may involve multiple group aggregations,
    # the overall benefit of vectorized operations often outweighs this.

In [85]:
# Grouped Time Resampling

# For time series data, the resample method is semantically a group operation based on a time intervalization.

# Here’s a small example table:


N = 15

times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)

df = pd.DataFrame({'time': times, 'value': np.arange(N)})

df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [86]:
# Here, we can index by 'time' and then resample:

df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [87]:
df.set_index('time').resample('2min').count()

,value
time,
2017-05-20 00:00:00,2
2017-05-20 00:02:00,2
2017-05-20 00:04:00,2
2017-05-20 00:06:00,2
2017-05-20 00:08:00,2
2017-05-20 00:10:00,2
2017-05-20 00:12:00,2
2017-05-20 00:14:00,1


In [88]:
#Suppose that a DataFrame contains multiple time series, marked by an additional group key column:

df2 = pd.DataFrame({'time': times.repeat(3), 'key': np.tile(['a', 'b', 'c'], N), 'value': np.arange(N * 3.)})


df2[:7]

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0


In [89]:
df2

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0
7,2017-05-20 00:02:00,b,7.0
8,2017-05-20 00:02:00,c,8.0
9,2017-05-20 00:03:00,a,9.0


In [105]:
# To do the same resampling for each value of 'key', we introduce the pandas.TimeGrouper object:

time_key = pd.TimeGrouper('5min')

AttributeError: module 'pandas' has no attribute 'TimeGrouper'

In [106]:
# We can then set the time index, group by 'key' and time_key, and aggregate:

resampled = (df2.set_index('time').groupby(['key', time_key]).sum())
resampled

NameError: name 'time_key' is not defined

In [107]:
resampled.reset_index()

NameError: name 'resampled' is not defined

In [ ]:
# Techniques for Method Chaining

# When applying a sequence of transformations to a dataset, you may find yourself creating numerous 
    # temporary variables that are never used in your analysis. 

# Consider this example, for instance:
    
df = load_data()
df2 = df[df['col2'] < 0]
df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()
result = df2.groupby('key').col1_demeaned.std()

In [ ]:
# While we’re not using any real data above, the example highlights some new methods.
    # First, the DataFrame.assign method is a functional alternative to column assignments of 
    # the form df[k] = v. 
    # Rather than modifying the object in-place, it returns a new DataFrame with the indicated modifications.
    # So these statements are equivalent:

# Usual non-functional way
df2 = df.copy()
df2['k'] = v

# Functional assign way
df2 = df.assign(k=v)

In [ ]:
# Assigning in-place may execute faster than using assign, but assign enables easier method chaining:

result = df2.assign(col1_demeaned=df2.col1 - df2.col2.mean()).groupby('key').col1_demeaned.std()

In [ ]:
# One thing to keep in mind when doing method chaining is that you may need to refer to temporary objects.
    # In the preceding example, we cannot refer to the result of load_data until it has been assigned
    # # to the temporary variable df. 
    # To help with this, assign and many other pandas functions accept function-like arguments,
    # also known as callables.

# To show callables in action, consider a fragment of the example from before:

    
df = load_data()
df2 = df[df['col2'] < 0]

In [ ]:
# This can be rewritten as:
    
df = load_data()[lambda x: x['col2'] < 0]

In [ ]:
# Above, the result of load_data is not assigned to a variable,
    # so the function passed into [] is then bound to the object at that stage of the method chain.

# We can continue, then, and write the entire sequence as a single chained expression:

result = load_data()[lambda x: x.col2 < 0].
          assign(col1_demeaned=lambda x: x.col1 - x.col1.mean()).
          groupby('key').col1_demeaned.std()

In [ ]:
# Whether you prefer to write code in this style is a matter of taste, and splitting up the
    # expression into multiple steps may make your code more readable.

In [ ]:
# The pipe Method

# You can accomplish a lot with built-in pandas functions and the approaches to 
    # method chaining with callables that we just looked at.
    # However, sometimes you need to use your own functions or functions from third-party libraries. 
    # This is where the pipe method comes in.

# Consider a sequence of function calls:

a = f(df, arg1=v1)
b = g(a, v2, arg3=v3)
c = h(b, arg4=v4)

In [ ]:
# When using functions that accept and return Series or DataFrame objects,
    # you can rewrite this using calls to pipe:

result = df.pipe(f, arg1=v1).pipe(g, v2, arg3=v3).pipe(h, arg4=v4)

In [ ]:
# The statement f(df) and df.pipe(f) are equivalent, but pipe makes chained invocation easier.

# A potentially useful pattern for pipe is to generalize sequences of operations into reusable functions.
    # As an example, let’s consider substracting group means from a column:

g = df.groupby(['key1', 'key2'])
df['col1'] = df['col1'] - g.transform('mean')

In [ ]:
# Suppose that you wanted to be able to demean more than one column and easily change the group keys.
    # Additionally, you might want to perform this transformation in a method chain.

# Here is an example implementation:

def group_demean(df, by, cols):
    result = df.copy()
    g = df.groupby(by)
    for c in cols:
        result[c] = df[c] - g[c].transform('mean')
    return result

In [ ]:
# Then it is possible to write:

result = df[df.col1 < 0].pipe(group_demean, ['key1', 'key2'], ['col1'])

# The End